In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

In [5]:
playercount_df = pd.DataFrame([{'Total Players':len(purchase_data['SN'].unique())}])
playercount_df

,Total Players
0,576


## Purchasing Analysis (Total)

In [6]:
purchasingtotal_df = pd.DataFrame([{'Number of Unique Items':len(purchase_data['Item ID'].unique()),
                                    'Average Price':purchase_data['Price'].mean(),
                                    'Number of Purchases':len(purchase_data['Purchase ID']),
                                    'Total Revenue':purchase_data['Price'].sum()}])

purchasingtotal_df['Average Price'] = purchasingtotal_df['Average Price'].map('${:,.2f}'.format)
purchasingtotal_df['Total Revenue'] = purchasingtotal_df['Total Revenue'].map('${:,.2f}'.format)

purchasingtotal_df = purchasingtotal_df[['Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue']]

purchasingtotal_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [7]:
playergenders_df = purchase_data[['SN', 'Gender']].drop_duplicates(subset='SN', keep='first', inplace=False)
genderdemo_df = pd.DataFrame(playergenders_df['Gender'].value_counts()).rename(columns={'Gender' : 'Total Count'})

genderdemo_df['Percentage of Players'] = genderdemo_df['Total Count'] / len(playergenders_df.index) * 100
genderdemo_df['Percentage of Players'] = genderdemo_df['Percentage of Players'].map('{:,.2f}'.format)

genderdemo_df

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

In [8]:
genderprice_group = purchase_data[['Gender', 'Price']].groupby(['Gender'])
purchasinggender_df = genderprice_group.count().rename(columns={'Price':'Purchase Count'})

purchasinggender_df['Average Purchase Price']  = genderprice_group.mean()
purchasinggender_df['Total Purchase Value']  = genderprice_group.sum()
purchasinggender_df['Avg Total Purchase per Person'] = purchasinggender_df.loc[purchasinggender_df.index, 'Total Purchase Value'] / genderdemo_df.loc[purchasinggender_df.index, 'Total Count']

purchasinggender_df['Average Purchase Price'] = purchasinggender_df['Average Purchase Price'].map('${:,.2f}'.format)
purchasinggender_df['Total Purchase Value'] = purchasinggender_df['Total Purchase Value'].map('${:,.2f}'.format)
purchasinggender_df['Avg Total Purchase per Person'] = purchasinggender_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

purchasinggender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [9]:
playerages_df = purchase_data[['SN', 'Age']].drop_duplicates(subset='SN', keep='first', inplace=False)
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, playerages_df['Age'].max()]
age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
playerages_df['Age Group'] = pd.DataFrame(pd.cut(playerages_df['Age'], age_bins, labels=age_groups))

agedemo_df = playerages_df.groupby('Age Group').count().drop(columns='SN').rename(columns={'Age':'Total Count'}).reindex(age_groups)
agedemo_df['Percentage of Players'] = agedemo_df['Total Count'] / agedemo_df['Total Count'].sum() * 100
agedemo_df['Percentage of Players'] = agedemo_df['Percentage of Players'].map('{:,.2f}'.format)

agedemo_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

In [10]:
ageprice_df = purchase_data[['SN', 'Age', 'Price']]
ageprice_df['Age Group'] = pd.DataFrame(pd.cut(ageprice_df['Age'], age_bins, labels=age_groups))
ageprice_group = ageprice_df.groupby('Age Group')

purchasingage_df = ageprice_group.count().drop(columns=['SN', 'Price']).rename(columns={'Age':'Total Count'}).reindex(age_groups)
purchasingage_df['Average Purchase Price'] = ageprice_group.mean().drop(columns='Age')
purchasingage_df['Total Purchase Value'] = ageprice_group.sum().drop(columns='Age')
purchasingage_df['Avg Total Purchase per Person'] = purchasingage_df.loc[purchasingage_df.index, 'Total Purchase Value'] / agedemo_df.loc[purchasingage_df.index, 'Total Count']

purchasingage_df['Average Purchase Price'] = purchasingage_df['Average Purchase Price'].map('${:,.2f}'.format)
purchasingage_df['Total Purchase Value'] = purchasingage_df['Total Purchase Value'].map('${:,.2f}'.format)
purchasingage_df['Avg Total Purchase per Person'] = purchasingage_df['Avg Total Purchase per Person'].map('${:,.2f}'.format)

purchasingage_df

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Total Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [11]:
playerprice_group = purchase_data[['SN', 'Price']].groupby('SN')
topspenders_df = playerprice_group.count().rename(columns={'Price':'Purchase Count'})

topspenders_df['Average Purchase Price']  = playerprice_group.mean()
topspenders_df['Total Purchase Value']  = playerprice_group.sum()

topspenders_df = topspenders_df.sort_values(by='Total Purchase Value', ascending=False)

topspenders_df['Average Purchase Price'] = topspenders_df['Average Purchase Price'].map('${:,.2f}'.format)
topspenders_df['Total Purchase Value'] = topspenders_df['Total Purchase Value'].map('${:,.2f}'.format)

topspenders_df.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [12]:
item_group = purchase_data[['Item ID', 'Item Name', 'Price']].groupby(['Item ID', 'Item Name'])
mostpopularitems_df = item_group.count().sort_values(by='Price', ascending=False).rename(columns={'Price':'Purchase Count'})

mostpopularitems_df['Item Price'] = item_group.max()
mostpopularitems_df['Total Purchase Value'] = item_group.sum()

mostpopularitems_df['Item Price'] = mostpopularitems_df['Item Price'].map('${:,.2f}'.format)
mostpopularitems_df['Total Purchase Value'] = mostpopularitems_df['Total Purchase Value'].map('${:,.2f}'.format)

mostpopularitems_df.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [16]:
mostprofitableitems_df = mostpopularitems_df.copy()

mostprofitableitems_df['Total Purchase Value'] = mostprofitableitems_df['Total Purchase Value'].str.replace('$', '').astype(float)

mostprofitableitems_df = mostprofitableitems_df.sort_values(by='Total Purchase Value', ascending=False)

mostprofitableitems_df['Total Purchase Value'] = mostprofitableitems_df['Total Purchase Value'].map('${:,.2f}'.format)

mostprofitableitems_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


## Observable Trends

* The majority of the playerbase is male (84.03\%). Furthermore, males contributed the most to the game's revenue (\\$1,967.64) compared to females (\\$361.94) and other/non-disclosed (\\$50.19).

* The greatest age demographic is 20 to 24 years old (44.79\%). Younger demographics as well as older demographics are decreasingly smaller and follow a bell curve.

* The most proftable items do not exceed \\$5.00. Players probably consider anything over that price to be too expensive for an item.